<i>Adapted from Recommenders ALS example</i>

# Running ALS on MIND (with PySpark)

Matrix factorization by [ALS](https://spark.apache.org/docs/latest/api/python/_modules/pyspark/ml/recommendation.html#ALS) (Alternating Least Squares) is a well known collaborative filtering algorithm.

This notebook provides an example of how to utilize and evaluate ALS PySpark ML (DataFrame-based API) implementation, meant for large-scale distributed datasets.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType

from recommenders.utils.timer import Timer
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

from tempfile import TemporaryDirectory
from recommenders.datasets.mind import download_mind
from recommenders.datasets.download_utils import unzip_file

print(f"System version: {sys.version}")
print("Spark version: {}".format(pyspark.__version__))


System version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
Spark version: 3.5.4


Set the default parameters.

In [2]:
# top k items to recommend
TOP_K = 10

# MIND sizes: "demo", "small", or "large"
mind_type = 'demo'

# Column names for the dataset
COL_USER = "user_id"
COL_ITEM = "news_id"
COL_RATING = "rating"

### 0. Set up Spark context & directory

The following settings work well for debugging locally on VM - change when running on a cluster. We set up a giant single executor with many threads and specify memory cap. 

In [3]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap
spark = start_or_get_spark("ALS PySpark", memory="16g")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

25/04/06 09:29:45 WARN Utils: Your hostname, sondre-ThinkPad-E580 resolves to a loopback address: 127.0.1.1; using 10.21.36.87 instead (on interface enx6c02e0d7834b)
25/04/06 09:29:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/06 09:29:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/04/06 09:30:03 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# Setup data storage location

tmpdir = TemporaryDirectory()
data_path = tmpdir.name
train_zip, valid_zip = download_mind(size=mind_type, dest_path=data_path)
unzip_file(train_zip, os.path.join(data_path, 'train'), clean_zip_file=False)
unzip_file(valid_zip, os.path.join(data_path, 'valid'), clean_zip_file=False)
train_behaviors_path = os.path.join(data_path, "train", "behaviors.tsv")

100%|██████████| 17.0k/17.0k [00:08<00:00, 1.99kKB/s]
100%|██████████| 9.84k/9.84k [00:03<00:00, 2.78kKB/s]


### 1. Download the MIND dataset

In [5]:
# Schema for behaviors.tsv
schema = StructType([
    StructField("impression_id", StringType(), True),  # Ignored for ALS
    StructField(COL_USER, StringType(), True),  # Will be converted later
    StructField("timestamp", StringType(), True),  # Convert to long if needed
    StructField("history", StringType(), True),  # List of past clicked news
    StructField("impressions", StringType(), True)  # Needs to be split into news_id + rating
])

# Load raw behaviors.tsv
data = (
    spark.read.option("sep", "\t").option("header", "false")
    .schema(schema)
    .csv(train_behaviors_path)
)

# Split impressions column ("n4-1 n5-0 n6-1") into separate rows
data = data.withColumn("impressions", F.explode(F.split(F.col("impressions"), " ")))

# Extract news_id and click status (e.g., "n4-1" → news_id="n4", rating=1)
data = data.withColumn(COL_ITEM, F.split(F.col("impressions"), "-")[0])
data = data.withColumn(COL_RATING, F.split(F.col("impressions"), "-")[1].cast(IntegerType()))

# Convert user_id and news_id to integers (ALS requires numeric IDs)
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.types import IntegerType

# Extract numeric part from IDs like "U123" or "N456"
data = data.withColumn(COL_USER, regexp_extract(col(COL_USER), r"\d+", 0).cast(IntegerType()))
data = data.withColumn(COL_ITEM, regexp_extract(col(COL_ITEM), r"\d+", 0).cast(IntegerType()))

# Drop unnecessary columns
data = data.select(COL_USER, COL_ITEM, COL_RATING)

# Count and remove articles / users with few interactions
user_counts = data.groupBy(COL_USER).count().filter(F.col("count") >= 10)
news_counts = data.groupBy(COL_ITEM).count().filter(F.col("count") >= 10)

data = data.join(user_counts, "user_id").join(news_counts, "news_id")


# Show transformed data
data.show()
data.groupBy(COL_RATING).count().show()


+-------+-------+------+-----+-----+
|news_id|user_id|rating|count|count|
+-------+-------+------+-----+-----+
|  13390|  82271|     0|  162| 1893|
|   7180|  82271|     0|  162|  725|
|  20785|  82271|     0|  162|  881|
|   6937|  82271|     0|  162| 1269|
|  15776|  82271|     0|  162|  604|
|  25810|  82271|     0|  162|  391|
|  20820|  82271|     0|  162|  590|
|  27294|  82271|     0|  162|  513|
|  18835|  82271|     0|  162|  864|
|  16945|  82271|     0|  162| 1305|
|   7410|  82271|     0|  162|   31|
|  23967|  82271|     0|  162|  937|
|  22679|  82271|     0|  162|  255|
|  20532|  82271|     0|  162|  484|
|  26651|  82271|     0|  162| 1523|
|  22078|  82271|     0|  162| 1486|
|   4098|  82271|     0|  162|  708|
|  16473|  82271|     0|  162| 1195|
|  13841|  82271|     0|  162| 1387|
|  15660|  82271|     0|  162|  639|
+-------+-------+------+-----+-----+
only showing top 20 rows



+------+------+
|rating| count|
+------+------+
|     1| 33306|
|     0|790435|
+------+------+



### 2. Split the data using the Spark random splitter provided in utilities

In [7]:
train, test = spark_random_split(data, ratio=0.75, seed=123)
print ("N train", train.cache().count())
print ("N test", test.cache().count())

25/04/06 09:40:07 WARN CacheManager: Asked to cache already cached data.


N train 618004


25/04/06 09:40:07 WARN CacheManager: Asked to cache already cached data.


N test 205737


### 3. Train the ALS model on the training data, and get the top-k recommendations for our testing data

To article interactions movie ratings, we use the rating data in the training set as users' explicit feedback.

In [8]:
header = {
    "userCol": COL_USER,
    "itemCol": COL_ITEM,
    "ratingCol": COL_RATING,
}


als = ALS(
    rank=50,
    maxIter=15,
    implicitPrefs=True,
    regParam=0.01,
    coldStartStrategy='drop',
    nonnegative=True,
    seed=42,
    alpha=45,
    **header
)

In [9]:
with Timer() as train_time:
    model = als.fit(train)

print(f"Took {train_time.interval} seconds for training.")

25/04/06 09:40:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Took 23.569571076000102 seconds for training.


In the movie recommendation use case, recommending movies that have been rated by the users do not make sense. Therefore, the rated movies are removed from the recommended items.

In order to achieve this, we recommend all movies to all users, and then remove the user-movie pairs that exist in the training dataset.

In [10]:
with Timer() as test_time:

    # Get the cross join of all user-item pairs and score them.
    users = train.select(COL_USER).distinct()
    items = train.select(COL_ITEM).distinct()
    user_item = users.crossJoin(items)
    dfs_pred = model.transform(user_item)

    # Remove seen items.
    dfs_pred_exclude_train = dfs_pred.alias("pred").join(
        train.alias("train"),
        (dfs_pred[COL_USER] == train[COL_USER]) & (dfs_pred[COL_ITEM] == train[COL_ITEM]),
        how='outer'
    )

    top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train[f"train.{COL_RATING}"].isNull()) \
        .select('pred.' + COL_USER, 'pred.' + COL_ITEM, 'pred.' + "prediction")

    # In Spark, transformations are lazy evaluation
    # Use an action to force execute and measure the test time 
    top_all.cache().count()

print(f"Took {test_time.interval} seconds for prediction.")

25/04/06 09:40:50 WARN Column: Constructing trivially true equals predicate, 'user_id#33 = user_id#33'. Perhaps you need to use aliases.
25/04/06 09:40:50 WARN Column: Constructing trivially true equals predicate, 'news_id#41 = news_id#41'. Perhaps you need to use aliases.


Took 278.7115564999999 seconds for prediction.


In [11]:
top_all.show()

+-------+-------+------------+
|user_id|news_id|  prediction|
+-------+-------+------------+
|     17|   1538|  0.15820055|
|     17|   3172|         0.0|
|     17|   4803|0.0012495205|
|     17|  11644|  0.01418963|
|     17|  11821|         0.0|
|     17|  16109|0.0051098084|
|     17|  21503|         0.0|
|     17|  24459|         0.0|
|     17|  24768|         0.0|
|     17|  25311|         0.0|
|     17|  26103|0.0034035179|
|     17|  27784|  0.04035833|
|     22|    947|  0.02954644|
|     22|   1810|         0.0|
|     22|   2157| 0.019164255|
|     22|   2740|         0.0|
|     22|   5823|         0.0|
|     22|  10213|         0.0|
|     22|  16362|         0.0|
|     22|  16810|         0.0|
+-------+-------+------------+
only showing top 20 rows



### 4. Evaluate how well ALS performs

In [12]:
rank_eval = SparkRankingEvaluation(test, top_all, k = TOP_K, col_user=COL_USER, col_item=COL_ITEM, 
                                    col_rating=COL_RATING, col_prediction="prediction", 
                                    relevancy_method="top_k")

In [ ]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.024824
NDCG:	0.063141
Precision@K:	0.056736
Recall@K:	0.019012


### 5. Evaluate rating prediction

In [14]:
# Generate predicted ratings.
prediction = model.transform(test)
prediction.cache().show()


+-------+-------+------+-----+-----+------------+
|news_id|user_id|rating|count|count|  prediction|
+-------+-------+------+-----+-----+------------+
|    294|  15790|     0|  262| 1421|0.0065011675|
|    425|  38758|     1|   61| 2066|         0.0|
|    431|  43935|     1|  335|   97|         0.0|
|    533|  21700|     0|  133|   82| 0.052334305|
|    657|  43935|     0|  335|  101|         0.0|
|    712|  12027|     0|  235|  152|0.0014256807|
|    824|  78120|     0|  258|  928|0.0021753795|
|   1132|  12027|     0|  235| 1966|  0.78306013|
|   1138|  12027|     0|  235|  107|  0.08100026|
|   1636|  78120|     0|  258|  309|  0.11024747|
|   2022|  12027|     0|  235|  303| 8.478404E-4|
|   2581|  21700|     0|  133|   14|         0.0|
|   2770|  78120|     0|  258| 1010|0.0037514006|
|   2864|  19079|     1|   81|  944| 0.038738415|
|   2864|  57039|     0|   30|  944|         0.0|
|   2916|  12027|     0|  235|  666|         0.0|
|   2974|  19079|     0|   81| 1310| 0.019264665|


In [ ]:
rating_eval = SparkRatingEvaluation(test, prediction, col_user=COL_USER, col_item=COL_ITEM, 
                                    col_rating=COL_RATING, col_prediction="prediction")

print("Model:\tALS rating prediction",
      "RMSE:\t%f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(), sep='\n')

Model:	ALS rating prediction
RMSE:	0.317041
MAE:	0.167151
Explained variance:	-1.418830
R squared:	-1.719973


In [16]:
# cleanup spark instance and clear temp directory
spark.stop()
tmpdir.cleanup()

### 6. Changes over time

#### Attempt 0 - no modifications

Model:	ALS
Top K:	10
MAP:	0.000041
NDCG:	0.000069
Precision@K:	0.000020
Recall@K:	0.000041

Model:	ALS rating prediction
RMSE:	0.194057
MAE:	0.047414
Explained variance:	0.019782
R squared:	-0.002237

These extremely low metrics might be due to the dataset mostly being made up of non-interactions. Possible changes:
- Remove articles with low engagement
- Tune hyperparameters: increase rank, decrease regParam, and change alpha
- Convert data to implicit feedback

#### Attempt 1 - adding atmpt. 0 suggestions

Model:	ALS
Top K:	10
MAP:	0.026334
NDCG:	0.066722
Precision@K:	0.058610
Recall@K:	0.020850

Model:	ALS rating prediction
RMSE:	0.317204
MAE:	0.167511
Explained variance:	-1.369951
R squared:	-1.660865

Much better, but still very low. Might attempt a different method before continuing. 
